# OpenAI call to generate the Verhaal Speciaal story

In this notebook we will create a 'Verhaal Speciaal' story.

The story will be generated by a LLM (GPT-4) based on a prompt. 

The story will be based on user input. One of the user inputs is the reading level, which is based on class/group. 

In the last part of the notebook we can evaluate the level of the generated text.

### Contents
0. Installs and imports
1. Settings and prompt
2. Generate chapter one
3. Generate chapter two & three
4. Convert story to JSON

## 0. Installs and imports

In [1]:
#!pip install openai --upgrade

In [2]:
import openai
openai.__version__

'1.54.5'

In [3]:
#import the config file with the openai API key
import config

## 1. Settings

### Setting the reading levels
Four different reading levels have been defined, see below.

Both characters have their own reading level as Verhaal Speciaal is meant to be a reading combination for parent and child. Example: parent can have reading level 3, while the child can have reading level 1.  All combinations are possible. 

### User input

The user_input prompt collects the input the user of the story creates. This is taken from the javascript code of the original Verhaal Speciaal:
1. personage een        => character_one
2. personage twee       => character_two
3. wat                  => plot
4. waarom               => reasoning
5. waar                 => setting
6. wanneer              => time

And we set the reading levels:
7. klas/groep           => groep (will be mapped to reading_level)
8. Leesniveau ouder     => reading level

### Building the prompt 

The reading levels and user input are then used to build up a prompt..

We will generate a prompt consisting of three 'sub-prompts':

prompt =  basic_prompt + previous_text + chapter_prompt 

**basic_prompt**

The basic prompt sets the structure of the story. It defines there are two characters and a story teller.  It makes sure the story follows a pattern.  

**previous_text**

Only used for chapters 2 and 3. 
Input of the previous chapter(s): one or two. 

**chapter_prompt**

This are the chapter specific inputs:

1. Start new story, introduce characters, leave room for chapters 2 and 3
2. Follow up on chapter 1, use previous text and leave room for chapter 3
3. chapter 3: Final chapter, end the story, use previous text.

### Reading levels

In [4]:
#variables based on the reading level settings
level_one = 'Leesniveau 1: Tekst bestaat uit korte woorden die je precies zo schrijft zoals je ze uitspreekt. Voorbeelden hiervan zijn maan, bos, man, roos. Er mogen dus geen woorden voorkomen met bijvoorbeeld sch- en -ng en -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. De zinnen zijn zo kort mogelijk. Elke zin begint op een nieuwe regel. Er komen geen in hoofdletters voor, dus alle woorden worden met kleine letters geschreven.'

level_two = 'Leesniveau 2: Tekst bestaat uit één- en tweelettergrepige woorden. Woorden met -sch en -ng mogen voorkomen.  Woorden mogen voorkomen eindigend op -nk, -b, -d(t), -ch(t), -ooi, -aai, -oei, -eeuw, -ieuw, -uw. Een-. Ook mogen tweelettergrepige woorden met twee of drie medeklinkers na elkaar zoals staart, botst, sprong, bankje, knappe, winkel  Er mogen ook enkele makkelijke drielettergrepige woorden voorkomen, zoals sinterklaas. De zinnen zijn meestal kort. Soms mogen samengestelde zinnen voorkomen. Hoofdletters worden gebruikt.'

level_three = 'Leesniveau 3: Er mogen woorden gebruikt worden met 3 of meer lettergrepen. De zinnen mogen langer zijn. De zinnen kunnen bestaan uit een hoofdzin met een bijzin. Gebruik van leenwoorden, zoals bureau, horloge, chauffeur is beperkt toegestaan. Ook eenvoudige leesmoeilijkheden van leenwoorden komen voor: i en y uitgesproken als ie; c uitgesproken als k of als s. Zinnen mogen beginnen op dezelfde regel. Samengestelde zinnen mogen voorkomen. Hoofdletters worden gebruikt.'

level_four = 'Leesniveau 4: Er zijn geen beperkingen in woorden en zinslengte. Lastig te lezen leenwoorden (gamen), onbekende woorden (ov-pas, ornament) en leestekens (ideeën, ruïne) komen meer voor. Woorden eindigend op -ele, -eaal, -ueel, -iaal of -ieel komen voor. Ook woorden met een trema komen voor. Ook woorden beginnend met /ch/ uitgesproken als /sj/, eindigend op –ge, uitgesproken als /zje/, eindigend op –isch, woorden met klinkerreeks, leenwoorden met eau, é of è. Hoofdletters worden gebruikt.'


### User input

In [5]:
#These are the variables from the front end about the story . 
character_one = 'eddy'
character_two = 'jan'
plot ='een wandeling'
reasoning = 'ze verdwalen'
setting = 'in het bos' #waar
time = 'in de zomer'

# These are the input variables from the front end for the reading level
group_child = 7 #class the child is in 3,4,5,6,7,8
reading_level_parent = level_four # 1 2 3 4 based on reading level settings 

In [6]:
#Reading level conversion table CHILD

group = group_child #class the child is in 3,4,5,6,7,8

if group < 4:
    reading_level_child = level_one
    print(reading_level_child)
elif group == 4:
    reading_level_child = level_two
    print(reading_level_child)
elif group <= 6:
    reading_level_child = level_three
    print(reading_level_child)
elif group <= 8:
    reading_level_child = level_four
    print(reading_level_child)


Leesniveau 4: Er zijn geen beperkingen in woorden en zinslengte. Lastig te lezen leenwoorden (gamen), onbekende woorden (ov-pas, ornament) en leestekens (ideeën, ruïne) komen meer voor. Woorden eindigend op -ele, -eaal, -ueel, -iaal of -ieel komen voor. Ook woorden met een trema komen voor. Ook woorden beginnend met /ch/ uitgesproken als /sj/, eindigend op –ge, uitgesproken als /zje/, eindigend op –isch, woorden met klinkerreeks, leenwoorden met eau, é of è. Hoofdletters worden gebruikt.


In [7]:
#summarizing the reading levels
print(f'Reading level child: {reading_level_child[:12]}')
print(f'Reading level parent: {reading_level_parent[:12]}')

Reading level child: Leesniveau 4
Reading level parent: Leesniveau 4


### basic_prompt

In [8]:
#update reading levels
basic_prompt_v3 =f'''Je bent een kinderboekenschrijver. 
Je schrijft een verhaal het Nederlands waarbij je de drie-hoofdstukken-structuur van een toneelstuk volgt.
Dit is een scriptdialoog tussen twee personages en er is een Verteller die de scène schetst. 

Het leesniveau van de verteller is {reading_level_parent}.

Er zijn twee karakers die ieder een eigen leesniveau hebben. Hierna volgen de regels per niveau. Daarna wordt aangegeven welk niveau ieder personage heeft. 
Dit is een beschrijving van personage {character_one}.
Het leesniveau van personage {character_one} is niveau {reading_level_child}, dus houd het taalgebruik op dat niveau voor dit personage. Gebruik hiervoor de omschrijving van de hiervoor genoemde niveuas
Dit is een beschrijving van personage {character_two}.
Het leesniveau van personage {character_two} is niveau {reading_level_parent}, dus houd het taalgebruik op dat niveau voor dit personage. Gebruik hiervoor de omschrijving van de hiervoor genoemde niveaus houdt het hoofdstuk bij twee zinnen per karakter.

De algemene verhaallijn is: {plot}.
Dit is de reden achter het verhaal: {reasoning}.
De setting van het verhaal is: {setting}.
De tijd waarin het verhaal zich afspeelt is: {time}.

Gebruik de volgende regels om te output te structureren:
Iedere zin of paragraaf van het verhaal moet bij de Verteller, {character_one} of {character_two} horen. 
De verteller wordt altijd aangeduid als Verteller. Gebruik het format Verteller | tekst
Voeg geen code tussen haakjes toe voor de Verteller.

Als een personage wat gaat vertellen, voeg {{char1}} of {{char2}} toe voor de naam van het personage dat spreekt.
voeg een | tussen alle woorden in zoals in dit voorbeeld: {{char1}} | {character_one} | tekst.
Aan het einde van het hoofdstuk moet de verteller een vraag stellen aan een van de personages over de voorgaande dialoog.
Aan het einde van het hoofdstuk moet de tekst '''"{ENDOFACT}"''' op een nieuwe regel worden toegevoegd.
Begin het hoofdstuk duidelijk met het nummer van het hoofdstuk. Bijvoorbeeld: 'Hoofdstuk 1'.
Zorg ervoor dat de personages hetzelfde blijven in de verschillende hoofdstukken en dat ze weten wat er gezegd is.
Voeg geen uitleg toe, alleen de dialoog.
Voeg geen nieuwe personages of settings toe aan de dialoog.
De allereerste regel van de tekst moet een gegenereerde titel zijn. Gebruik alleen letters en spaties, in de titel staat niet het woord 'titel'.
Gebruik geen speciale tekens in de tekst, alleen letters, spaties en nieuwe regels.
'''

### Previous text

In [9]:
#for chapter one empty, for chapter 2/3 will be updated, see below
previous = " " 

### Chapter prompt

In [10]:
chapter_1 = f'''Dit is het eerste hoofdstuk van drie, zorg dus dat het verhaal verder kan gaan.'''

In [11]:
chapter_2 = f'''Dit is het tweede hoofdstuk van drie. Ga door op het eerste hoofdstuk wat je uit deze tekst haalt: {previous}. Zorg dat het verhaal verder kan gaan in hoofdstuk 3.'''

In [12]:
chapter_3 = f'''Dit is het laatste hoofdstuk dus zorg voor een goed en happy einde. Ga door met het verhaal gebaseerd op hoofdstuk 1 en 2 wat je uit de deze tekst haalt: {previous}'''

### Concatenate prompt
We will use v3 as this is the 3rd version of the prompt (to keep it similar to original Verhaal Speciaal)

prompt_v3 = basic_prompt + chapter_prompt

In [13]:
prompt_v3_ch1 = basic_prompt_v3 + chapter_1 
print(prompt_v3_ch1)

Je bent een kinderboekenschrijver. 
Je schrijft een verhaal het Nederlands waarbij je de drie-hoofdstukken-structuur van een toneelstuk volgt.
Dit is een scriptdialoog tussen twee personages en er is een Verteller die de scène schetst. 

Het leesniveau van de verteller is Leesniveau 4: Er zijn geen beperkingen in woorden en zinslengte. Lastig te lezen leenwoorden (gamen), onbekende woorden (ov-pas, ornament) en leestekens (ideeën, ruïne) komen meer voor. Woorden eindigend op -ele, -eaal, -ueel, -iaal of -ieel komen voor. Ook woorden met een trema komen voor. Ook woorden beginnend met /ch/ uitgesproken als /sj/, eindigend op –ge, uitgesproken als /zje/, eindigend op –isch, woorden met klinkerreeks, leenwoorden met eau, é of è. Hoofdletters worden gebruikt..

Er zijn twee karakers die ieder een eigen leesniveau hebben. Hierna volgen de regels per niveau. Daarna wordt aangegeven welk niveau ieder personage heeft. 
Dit is een beschrijving van personage eddy.
Het leesniveau van personage ed

## 2. Generate chapter one

In [14]:
#store your openai api key as an environment variable - as required by OpenAI
import os
import config
os.environ['OPENAI_API_KEY'] = config.OpenAI_key # replace with your API-key, but don't keep it in your source code :-)

client = openai.Client()

In [15]:
#Function to call the OpenAI API
def create_chat_completion(prompt, model="gpt-4"):
  
    # Create the chat completion
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
            ],
        model=model,
    )

    # Return the generated response
    chapter = chat_completion.choices[0].message.content
    return chapter


In [16]:
chapter_one = create_chat_completion(prompt_v3_ch1)
print(chapter_one)

Titel: Verdwaald op een Zomerse Wandeling

Hoofdstuk 1

Verteller | Het is een stralende zomerdag en eddy en jan vertrekken voor een avontuurlijke wandeling in het bos.

{char1} | eddy | Het is ideaal wandelweer, niet? De zon schijnt en er hangt een frisse geur van dennen in de lucht.

{char2} | jan | Ja, inderdaad. Het is perfect buiten, laten we op pad gaan.

Verteller | De twee vrienden vervolgen hun wandeling, zich niet bewust van het onbekende pad dat voor hen ligt.

{char1} | eddy | Jan, weet je zeker dat we de juiste richting op gaan? Deze route lijkt niet op de kaart te staan.

{char2} | jan | Ik ben er vrij zeker van, eddy. Laten we gewoon doorlopen en zien waar we uitkomen.

Verteller | Terwijl de jongens verder wandelden, begonnen ze te beseffen dat ze verdwaald waren.

Verteller | eddy en jan, terwijl jullie genieten van de wandeling, lijkt het alsof jullie van het pad zijn afgedwaald, hoewel jullie dat nog steeds niet zeker weten. Gaan jullie proberen terug te gaan naar wa

## 3. Generate chapter two & three

The first chapter is input for chapter two and three. 



In [17]:
previous = f"Het vorige hoofdstuk was: {chapter_one}."

In [18]:
prompt_v3_ch2 = basic_prompt_v3+previous+chapter_2
print(prompt_v3_ch2)

Je bent een kinderboekenschrijver. 
Je schrijft een verhaal het Nederlands waarbij je de drie-hoofdstukken-structuur van een toneelstuk volgt.
Dit is een scriptdialoog tussen twee personages en er is een Verteller die de scène schetst. 

Het leesniveau van de verteller is Leesniveau 4: Er zijn geen beperkingen in woorden en zinslengte. Lastig te lezen leenwoorden (gamen), onbekende woorden (ov-pas, ornament) en leestekens (ideeën, ruïne) komen meer voor. Woorden eindigend op -ele, -eaal, -ueel, -iaal of -ieel komen voor. Ook woorden met een trema komen voor. Ook woorden beginnend met /ch/ uitgesproken als /sj/, eindigend op –ge, uitgesproken als /zje/, eindigend op –isch, woorden met klinkerreeks, leenwoorden met eau, é of è. Hoofdletters worden gebruikt..

Er zijn twee karakers die ieder een eigen leesniveau hebben. Hierna volgen de regels per niveau. Daarna wordt aangegeven welk niveau ieder personage heeft. 
Dit is een beschrijving van personage eddy.
Het leesniveau van personage ed

In [19]:
chapter_two = create_chat_completion(prompt_v3_ch2)
print(chapter_two)

Titel: Op Onbekend Terrein

Hoofdstuk 2

Verteller | Het bos was gevuld met de geluiden van vogels en het ruisen van de bladeren terwijl de zon vrolijk tussen de bomen scheen. Ondanks dat ze verdwaald waren, werd de stemming niet bedrukt.

{char1} | eddy | Hé, Jan, kijk eens naar die enorm rare gevormde boom daar. Ik heb zoiets nog nooit gezien.

{char2} | jan | Wow, die is inderdaad apart. Misschien maken we een ontdekking, Eddy, wie weet.

Verteller | Terwijl ze verder liepen, kwamen ze een plotselinge open plek tegen. De zonnestralen creëerden een bijna magische sfeer op de open plek.

{char1} | eddy | Moet je kijken, Jan! Dit lijkt wel een soort oase. Wat een prachtplek.

{char2} | jan | Je hebt helemaal gelijk, Eddy. Het is prachtig, maar we zijn nog steeds verdwaald.

Verteller | Ondanks het genieten van de wonderen rondom hen, begon het te dagen dat ze nog steeds geen idee hadden waar ze waren.

Verteller | Jan, jij merkte als eerste op dat jullie van het pad waren afgeraakt. De

In [20]:
#calling chapter trhee
previous = f"De vorige hoofdstukken waren {chapter_one} en {chapter_two}."
prompt_v3_ch3 = basic_prompt_v3+previous+chapter_3
chapter_three = create_chat_completion(prompt_v3_ch3)
print(chapter_three)

Titel: De Weg Terug naar Huis

Hoofdstuk 3

Verteller | De zon begon uiteindelijk te dalen en het werd duidelijk dat het tijd was om een oplossing te vinden.

{char1} | eddy | Jan, we kunnen hier niet de hele nacht blijven, ook al is deze plek adembenemend. We moeten een manier vinden om terug te komen.

{char2} | jan | Je hebt gelijk, Eddy. Laten we proberen terug te gaan naar die rare boom. Misschien dat we vanaf daar de weg terug kunnen vinden.

Verteller | Met hernieuwde hoop en enerige vervolgden zij hun weg terug. De bosgeluiden werden zachter, en het licht van de ondergaande zon gaf een geruststellende gloed.

{char1} | eddy | Kijk, daar is de boom! Ik herken de route weer. Ik denk dat we de weg terug hebben gevonden, Jan!

{char2} | jan | Geweldig, Eddy! Wat een avontuur hebben we beleefd. Ik kan niet wachten om dit alles aan onze vrienden te vertellen.

Verteller | In de eindeloze oranje gloed van de ondergaande zon vermengd met de groene schoonheid van het bos, vonden de twee

## 4. Save story as JSON

We need to create a .json file out of this story. This .json file will be analysed by the other scripts in the 'AVI Score' repository. 

The JSON structure is relatively simple, containing just one key-value pair. The complexity lies in the structured text content rather than in nested JSON objects or arrays.

The JSON file contains a single object with one key-value pair:
Key: "text"
Value: A long string containing a story

- First line contains the title
- Narrator sections: Paragraphs starting with "Verteller |"
- Character dialogues: Lines starting with "{char1} |" or "{char2} |"

Character dialogues follow this pattern:
- {char1} | anna | [dialogue text]
- {char2} | tom | [dialogue text]

Other things
- "{ENDOFACT}" appears at the end, likely indicating the end of a story act or section.
- The text uses newline characters (\n) to separate lines and sections.
- There are no nested objects or arrays within this JSON structure.
- The entire story is contained within a single string value.


In [21]:
story = {"text": chapter_one + chapter_two + chapter_three}
print(story)

{'text': 'Titel: Verdwaald op een Zomerse Wandeling\n\nHoofdstuk 1\n\nVerteller | Het is een stralende zomerdag en eddy en jan vertrekken voor een avontuurlijke wandeling in het bos.\n\n{char1} | eddy | Het is ideaal wandelweer, niet? De zon schijnt en er hangt een frisse geur van dennen in de lucht.\n\n{char2} | jan | Ja, inderdaad. Het is perfect buiten, laten we op pad gaan.\n\nVerteller | De twee vrienden vervolgen hun wandeling, zich niet bewust van het onbekende pad dat voor hen ligt.\n\n{char1} | eddy | Jan, weet je zeker dat we de juiste richting op gaan? Deze route lijkt niet op de kaart te staan.\n\n{char2} | jan | Ik ben er vrij zeker van, eddy. Laten we gewoon doorlopen en zien waar we uitkomen.\n\nVerteller | Terwijl de jongens verder wandelden, begonnen ze te beseffen dat ze verdwaald waren.\n\nVerteller | eddy en jan, terwijl jullie genieten van de wandeling, lijkt het alsof jullie van het pad zijn afgedwaald, hoewel jullie dat nog steeds niet zeker weten. Gaan jullie pr

In [25]:
import json
from datetime import datetime

# Convert the string into a JSON serializable format, e.g., as a dictionary
story_to_save = story

# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d_%H:%M")

# Create a filename with the current date
filename = f"V_S_{current_date}.json"
file_path = os.path.join('json', filename)

# Save the data to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(story_to_save, json_file, indent=4)

print(f"Data saved to {filename}")

Data saved to V_S_2024-12-04_15:35.json


## 5. Validate the new .json with an old example

In [23]:

#Here we use an old .json based on the Javascript code base

import json
from pprint import pprint

# Read the JSON file
with open('m1q9ewcov5mw22oftf8_1_1.json', 'r') as file:
    data = json.load(file)

# Pretty print using json.dumps()
print("Pretty printed using json.dumps():")
print(json.dumps(data, indent=4))

# Pretty print using pprint
print("\nPretty printed using pprint:")
pprint(data['text'])

FileNotFoundError: [Errno 2] No such file or directory: 'm1q9ewcov5mw22oftf8_1_1.json'

In [ ]:
#Here we show the new json.
import json
from pprint import pprint

# Read the JSON file
with open('V_S_2024-11-28_12:50.json', 'r') as file:
    data = json.load(file)

# Pretty print using json.dumps()
print("Pretty printed using json.dumps():")
print(json.dumps(data, indent=4))

# Pretty print using pprint
print("\nPretty printed using pprint:")
pprint(data)

Pretty printed using json.dumps():
{
    "text": "Titel: Een Wandeling die Verliep\n\nHoofdstuk 1\n\nVerteller | Jan en Eddy gaan naar het bos.\n\nVerteller | Het is een warme dag. \n\n{char1} | Eddy | Ik zie veel bomen. \n\n{char2} | Jan | Ja, ik ook. \n\nVerteller | Ze lopen op een pad in het bos.\n\n{char1} | Eddy | Waar gaat dit pad heen? \n\n{char2} | Jan | Ik weet het niet. \n\nVerteller | Jan en Eddy gaan verder op het pad.\n\nVerteller | Maar dan stopt het pad.\n\n{char1} | Eddy | Ik zie het pad niet meer. \n\n{char2} | Jan | We zijn weg!\n\nVerteller | Ze kijken rond. \n\nVerteller | Ze zien geen pad meer.\n\nVerteller | Jan, waar denk jij dat het pad is?\n\n{ENDOFACT}Titel: Verdwaald in het Bos\n\nHoofdstuk 2\n\nVerteller | Jan en Eddy zijn verdwaald in het bos.\n\nVerteller | Ze kijken rond, maar zien nergens een pad.\n\n{char1} | Eddy | Wat nu? \n\n{char2} | Jan | Uh, ik weet het niet. \n\nVerteller | Ze zien een groot meer.\n\nVerteller | Is dit het einde van het bos?\n\n{

## Evaluation of the differences

### logbook
 - change context to text (done)
 - remove 'Titel' from first line (option)
 - add {char1} or {char2} before name (done)
 - add {ENDOFACT} (done) 
 - 28/11 11:04 add to prompt the line: 'Het leesniveau van de verteller is {level_one }
 - 28/11 12:00 Nieuwe leesniveaus gedefinieerd, zie word doc voor toelichting.
 - 3/12: Leesniveuas verder aangepast. 
 
